In [27]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 15 11:01:32 2022

@author: Simone
"""

import numpy as np 
import pandas as pd 
from IPython.display import display
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import DBSCAN,KMeans
from sklearn.preprocessing import StandardScaler
import pyproj
import json
from shapely.geometry import Point, mapping
from functools import partial
from shapely.ops import transform
from mpl_toolkits.mplot3d import proj3d
from scipy.spatial.distance import cdist


%matplotlib notebook
pop = pd.read_csv("new/popSessoEtaLngLatSize2021.csv", dtype = {"ITTER107":str})
pop

,ITTER107,Territorio,Sesso,Età,Value,lng,lat,Size km2
0,1272,Torino,maschi,0 anni,2900,7.680687,45.073274,130.17
1,1272,Torino,femmine,0 anni,2724,7.680687,45.073274,130.17
2,1272,Torino,totale,0 anni,5624,7.680687,45.073274,130.17
3,1272,Torino,femmine,1 anni,2890,7.680687,45.073274,130.17
4,1272,Torino,maschi,1 anni,3015,7.680687,45.073274,130.17
...,...,...,...,...,...,...,...,...
2460235,IT111,Sud Sardegna,maschi,25 anni,1585,NaN,NaN,NaN
2460236,IT111,Sud Sardegna,femmine,91 anni,720,NaN,NaN,NaN
2460237,IT111,Sud Sardegna,femmine,23 anni,1330,NaN,NaN,NaN
2460238,IT111,Sud Sardegna,maschi,totale,169114,NaN,NaN,NaN


## Preprocessing

#### Get marital status

In [28]:
mar =  pd.read_csv("new/maritalStatus2020Agg2021.csv", dtype = {"ITTER107":str})
mar['ITTER107'] = mar['ITTER107'].str.lstrip('0')
mar = mar.astype({"Value": int})
mar

,ITTER107,Territorio,Sesso,Età,Stato civile,Value
0,IT,Italia,maschi,0 anni,nubile/celibe,212756
1,IT,Italia,maschi,0 anni,totale,212756
2,IT,Italia,femmine,0 anni,nubile/celibe,202218
3,IT,Italia,femmine,0 anni,totale,202218
4,IT,Italia,totale,0 anni,nubile/celibe,414974
...,...,...,...,...,...,...
11143435,41044,Pesaro,totale,totale,coniugata/o,44776
11143436,41044,Pesaro,totale,totale,divorziata/o,4109
11143437,41044,Pesaro,totale,totale,nubile/celibe,40387
11143438,41044,Pesaro,totale,totale,totale,96786


In [29]:
tot = pd.merge(pop, mar,how='outer',left_on=['ITTER107','Territorio','Sesso','Età'],right_on=['ITTER107','Territorio','Sesso','Età'],indicator=True)

In [30]:
tot = tot.drop(['Value_x','_merge'], axis=1)
tot= tot.rename(columns={'Value_y': 'Value'})
tot

,ITTER107,Territorio,Sesso,Età,lng,lat,Size km2,Stato civile,Value
0,1272,Torino,maschi,0 anni,7.680687,45.073274,130.17,nubile/celibe,3030
1,1272,Torino,maschi,0 anni,7.680687,45.073274,130.17,totale,3030
2,1272,Torino,femmine,0 anni,7.680687,45.073274,130.17,nubile/celibe,2905
3,1272,Torino,femmine,0 anni,7.680687,45.073274,130.17,totale,2905
4,1272,Torino,totale,0 anni,7.680687,45.073274,130.17,nubile/celibe,5935
...,...,...,...,...,...,...,...,...,...
11143435,IT111,Sud Sardegna,totale,20 anni,NaN,NaN,NaN,totale,2901
11143436,IT111,Sud Sardegna,totale,20 anni,NaN,NaN,NaN,divorziata/o,3
11143437,IT111,Sud Sardegna,totale,20 anni,NaN,NaN,NaN,coniugata/o,5
11143438,IT111,Sud Sardegna,totale,20 anni,NaN,NaN,NaN,nubile/celibe,2893


In [31]:
tot = tot.loc[tot['ITTER107'].str.isdigit()] #No areas
tot = tot.loc[tot['Età'] != "totale"]
# tot = tot.loc[tot['Sesso'] != "totale"]
# tot = tot.loc[tot['Stato civile'] != "totale"]
try:
    tot['Età'] = tot['Età'].map(lambda x: x.rstrip(' anni'))
    tot['Età'] = tot['Età'].map(lambda x: x.rstrip(' anni e più'))
except:
    print("Already done")
tot['Età'] = tot['Età'].astype(int)
tot["ITTER107"] = pd.to_numeric(tot["ITTER107"]) #Can do because no areas
tot

,ITTER107,Territorio,Sesso,Età,lng,lat,Size km2,Stato civile,Value
0,1272,Torino,maschi,0,7.680687,45.073274,130.17,nubile/celibe,3030
1,1272,Torino,maschi,0,7.680687,45.073274,130.17,totale,3030
2,1272,Torino,femmine,0,7.680687,45.073274,130.17,nubile/celibe,2905
3,1272,Torino,femmine,0,7.680687,45.073274,130.17,totale,2905
4,1272,Torino,totale,0,7.680687,45.073274,130.17,nubile/celibe,5935
...,...,...,...,...,...,...,...,...,...
10953548,75098,Presicce-Acquarica,femmine,97,18.263469,39.900977,43.06,totale,6
10953549,75098,Presicce-Acquarica,femmine,97,18.263469,39.900977,43.06,divorziata/o,0
10953550,75098,Presicce-Acquarica,femmine,97,18.263469,39.900977,43.06,nubile/celibe,1
10953551,75098,Presicce-Acquarica,femmine,97,18.263469,39.900977,43.06,vedova/o,5


### Get incomes

In [32]:
inc =  pd.read_csv("New/redditi2020modificato.csv")
# mar = mar.astype({"Value": int})
inc["Redditi 2020"] = inc["2020"]
inc.drop(["Comune","Regione","2020"], axis = 1, inplace= True)
tot = pd.merge(tot, inc ,how='inner',left_on=['ITTER107'],right_on=['Codice Comune formato alfanumerico'])
tot

,ITTER107,Territorio,Sesso,Età,lng,lat,Size km2,Stato civile,Value,Codice Comune formato alfanumerico,Redditi 2020
0,1272,Torino,maschi,0,7.680687,45.073274,130.17,nubile/celibe,3030,1272,22130
1,1272,Torino,maschi,0,7.680687,45.073274,130.17,totale,3030,1272,22130
2,1272,Torino,femmine,0,7.680687,45.073274,130.17,nubile/celibe,2905,1272,22130
3,1272,Torino,femmine,0,7.680687,45.073274,130.17,totale,2905,1272,22130
4,1272,Torino,totale,0,7.680687,45.073274,130.17,nubile/celibe,5935,1272,22130
...,...,...,...,...,...,...,...,...,...,...,...
10835008,75098,Presicce-Acquarica,femmine,97,18.263469,39.900977,43.06,totale,6,75098,9160
10835009,75098,Presicce-Acquarica,femmine,97,18.263469,39.900977,43.06,divorziata/o,0,75098,9160
10835010,75098,Presicce-Acquarica,femmine,97,18.263469,39.900977,43.06,nubile/celibe,1,75098,9160
10835011,75098,Presicce-Acquarica,femmine,97,18.263469,39.900977,43.06,vedova/o,5,75098,9160


### Add number of components in family

In [33]:
fam =  pd.read_csv("DCSS_FAM_POP_03052022114048969.csv",)
fam
fam = fam.loc[fam['ITTER107'].str.isdigit()] #No areas
fam["AvgNumberOfComponents"] = fam["Value"]
fam.drop(['Value','Flags','Flag Codes', 'TIME','Seleziona periodo',
          'Tipo dato','TIPO_DATO_CENS_POP','TIPO_DATO_CENS_POP','Territorio'], axis = 1, inplace = True)
fam["ITTER107"] = pd.to_numeric(fam["ITTER107"]) #Can do because no areas
tot = pd.merge(tot, fam ,how='inner',left_on=['ITTER107'],right_on=['ITTER107'])
tot.drop(['Codice Comune formato alfanumerico'], axis = 1, inplace = True)
tot
#Lose Borgo, Ville, Novella

,ITTER107,Territorio,Sesso,Età,lng,lat,Size km2,Stato civile,Value,Redditi 2020,AvgNumberOfComponents
0,1272,Torino,maschi,0,7.680687,45.073274,130.17,nubile/celibe,3030,22130,1.98
1,1272,Torino,maschi,0,7.680687,45.073274,130.17,totale,3030,22130,1.98
2,1272,Torino,femmine,0,7.680687,45.073274,130.17,nubile/celibe,2905,22130,1.98
3,1272,Torino,femmine,0,7.680687,45.073274,130.17,totale,2905,22130,1.98
4,1272,Torino,totale,0,7.680687,45.073274,130.17,nubile/celibe,5935,22130,1.98
...,...,...,...,...,...,...,...,...,...,...,...
10830895,75098,Presicce-Acquarica,femmine,97,18.263469,39.900977,43.06,totale,6,9160,2.37
10830896,75098,Presicce-Acquarica,femmine,97,18.263469,39.900977,43.06,divorziata/o,0,9160,2.37
10830897,75098,Presicce-Acquarica,femmine,97,18.263469,39.900977,43.06,nubile/celibe,1,9160,2.37
10830898,75098,Presicce-Acquarica,femmine,97,18.263469,39.900977,43.06,vedova/o,5,9160,2.37


In [34]:
bins= [0,3,13,20,30,40,50,60,70,80,90,100,110]
labels = ['Infant','Kid','Teen','20s','30s','40s','50s','60s','70s','80s','90s','100s or older']
tot['AgeGroup'] = pd.cut(tot['Età'], bins=bins, labels=labels, right=False) ##Create age groups

#Gropu by age group
df = tot.groupby(['Territorio','Sesso','Stato civile','AgeGroup'], as_index=True).agg(
    {'Value':'sum','lat': 'first','lng':'first', 'Size km2':'first',
     'Redditi 2020':'first','AvgNumberOfComponents':'first','ITTER107':'first'}).reset_index()

In [35]:
df

,Territorio,Sesso,Stato civile,AgeGroup,Value,lat,lng,Size km2,Redditi 2020,AvgNumberOfComponents,ITTER107
0,Abano Terme,femmine,coniugata/o,Infant,0,NaN,NaN,NaN,NaN,NaN,NaN
1,Abano Terme,femmine,coniugata/o,Kid,0,NaN,NaN,NaN,NaN,NaN,NaN
2,Abano Terme,femmine,coniugata/o,Teen,0,45.359387,11.788713,21.57,20753.0,2.22,28001.0
3,Abano Terme,femmine,coniugata/o,20s,84,45.359387,11.788713,21.57,20753.0,2.22,28001.0
4,Abano Terme,femmine,coniugata/o,30s,447,45.359387,11.788713,21.57,20753.0,2.22,28001.0
...,...,...,...,...,...,...,...,...,...,...,...
1420915,Zungri,totale,vedova/o,60s,23,38.656782,15.983636,23.26,8897.0,2.56,102050.0
1420916,Zungri,totale,vedova/o,70s,35,38.656782,15.983636,23.26,8897.0,2.56,102050.0
1420917,Zungri,totale,vedova/o,80s,50,38.656782,15.983636,23.26,8897.0,2.56,102050.0
1420918,Zungri,totale,vedova/o,90s,14,38.656782,15.983636,23.26,8897.0,2.56,102050.0


In [37]:
df.dropna().to_csv("datasetItalyComplete.csv", encoding='utf-8',index=False)